# Transfer Learning

In [1]:
import json
import sys
from functools import partial
from pathlib import Path

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

from src.Evaluation import get_confusion_matrix_for_model_and_data
from src.Helpers import get_project_root

sys.path.append("../src")

from tensorflow import keras
import tensorflow as tf
import numpy as np

from src.TrainProdecure import train_single_model
from src.ModelBuilder import get_FCN
from src.LoadData import get_fcn_datasets_test_train_np_arrays
from src.Helpers import (append_to_csv, get_project_root)

tf.config.list_physical_devices('GPU')

ModuleNotFoundError: No module named 'PyPDF2'

In [2]:
USED_DATASETS = get_fcn_datasets_test_train_np_arrays('../datasets/')
GENERATE_MODEL = partial(get_FCN, transfer_learning=True)
MODEL_NAME = 'Pre-Trained FCN'
PRE_TRAIN_EPOCHS = 30
FINE_TUNE_EPOCHS = 30
FINE_TUNE_LEARNING_RATE = 10e-3

CSV_PATH = str(get_project_root() / "results/transfer_learning.csv")
MODEL_PATH = str(get_project_root() / "models/transfer_learning")

In [4]:
def transfer_learning_model_one_dataset(x_train, y_train, dataset_name):
    input_size = x_train.shape[1]
    output_size = len(np.unique(y_train))

    model = GENERATE_MODEL(input_size, output_size)
    model.build(input_shape=(None, input_size, 1))

    return train_single_model(
        model, x_train, y_train,
        epochs=PRE_TRAIN_EPOCHS, batch_size=25, model_name='FCN',
        dataset_name=dataset_name,
        quiet=True
    )

In [ ]:
def fine_tune_model(pre_trained_model, x_train, y_train, x_test, y_test, dataset_name):
    input_size = x_train.shape[1]
    output_size = len(np.unique(y_train))
    headless_model_layers = pre_trained_model.layers[:-1]
    fine_tuned_model = keras.Sequential([*headless_model_layers, keras.layers.Dense(output_size, activation='softmax')])
    fine_tuned_model.build(input_shape=(None, input_size, 1))

    return train_single_model(
        fine_tuned_model, x_train, y_train, x_test, y_test,
        epochs=FINE_TUNE_EPOCHS, batch_size=25, model_name='FCN', dataset_name=dataset_name,
        quiet=True, learning_rate=FINE_TUNE_LEARNING_RATE
    )

In [ ]:
# Create some pretrained models
pretrained_models = []

for dataset_name, data_dict in tqdm(USED_DATASETS.items(), unit='datasets', desc='Transfer Learning for all datasets'):
    print(f"Training model targeting dataset {dataset_name}")

    x_train, y_train = data_dict['train_data']
    x_test, y_test = data_dict['test_data']
    pre_trained_model, pre_train_history = transfer_learning_model_one_dataset(                x_train, y_train, dataset_name)
    pretrained_models.append((pre_trained_model, dataset_name))

In [2]:
# Copy parameters of the pre-trained models and fine-tune them on the target datasets
model_path = str(get_project_root() / "models")
result_csv_path = str(get_project_root()) + "results/FCNTransferLearning_" + str(len(USED_DATASETS.items()))

if not (csv_path := Path(result_csv_path)).exists():
    csv_path.touch()
    append_to_csv(result_csv_path, ["Dataset", "Model", "Test Loss", "Test Accuracy", "Confusion Matrix", "History"])

for (pre_trained_model, pretrain_ds_name) in tqdm(pretrained_models, unit="models", desc="Fine-tuning models"):
    datasets_subset = {name: value for name, value in USED_DATASETS.items() if name != pretrain_ds_name}
    for dataset_name, data_dict in datasets_subset.items():
        print(f"Fine-tuning model on dataset {dataset_name}")

        x_train, y_train = data_dict['train_data']
        x_test, y_test = data_dict['test_data']

        # Fine-tuning:
        model, test_loss, test_acc, history = fine_tune_model(
            pre_trained_model, x_train, y_train, x_test, y_test, dataset_name
        )

        model_name = f"FCN pretrain-{pretrain_ds_name}"

        # save csv
        row = [
            dataset_name,
            model_name,
            test_loss,
            test_acc,
            json.dumps(
                get_confusion_matrix_for_model_and_data(
                    pre_trained_model, x_test, y_test
                ).tolist()
            ),
            json.dumps(history.history),
        ]
        append_to_csv(result_csv_path, row)
        # save model
        model_ds_path = model_path + "/" + dataset_name
        Path(model_ds_path).mkdir(exist_ok=True, parents=True)
        model.save(model_ds_path + "/" + model_name + ".h5")

NameError: name 'get_project_root' is not defined

In [7]:
def transfer_learning_for_all_datasets(used_datasets):
    """
    Function to train a model from scratch on a given dataset
    :return: the trained model, the loss, the accuracy, and the history
    """
    # Target ds for which it is fine-tuned:
    for dataset_name, data_dict in tqdm(used_datasets.items(), unit='datasets', desc='Transfer Learning for all datasets'):
        print(f"Training model targeting dataset {dataset_name}")

        x_train, y_train = data_dict['train_data']
        x_test, y_test = data_dict['test_data']

        # ds for pre-training:
        datasets_subset = {name: value for name, value in used_datasets.items() if name != dataset_name}
        for pre_train_ds_name, pre_train_data_dict in datasets_subset.items():
            print(f"Pre-training model on dataset {pre_train_ds_name}")

            pre_train_x_train, pre_train_y_train = pre_train_data_dict['train_data']
            pre_train_x_test, pre_train_y_test = pre_train_data_dict['test_data']

            # Pre-training a model:
            pre_trained_model, pre_train_history = transfer_learning_model_one_dataset(
                pre_train_x_train, pre_train_y_train, pre_train_x_test,
            )

            # Fine-tuning:
            _, fine_tuned_loss, fine_tuned_accuracy, fine_tuned_history = fine_tune_model(
                pre_trained_model, x_train, y_train, x_test, y_test, dataset_name
            )

            # Return results as a row in a csv file (a list):
            yield [
                    dataset_name,
                    pre_train_ds_name,
                    MODEL_NAME+f" (pre-trained on {pre_train_ds_name}; fine-tuned on {dataset_name})",
                    fine_tuned_loss,
                    fine_tuned_accuracy,
                    json.dumps(
                        get_confusion_matrix_for_model_and_data(
                            pre_trained_model, x_test, y_test
                        ).tolist()
                    ),
                    json.dumps(pre_train_history.history['accuracy'] + fine_tuned_history.history['accuracy']), # complete history of pre-training and fine-tuning
                ]


In [8]:
results = list(transfer_learning_for_all_datasets(USED_DATASETS))

Transfer Learning for all datasets:   0%|          | 0/17 [00:00<?, ?datasets/s]

Training model targeting dataset cbf
Pre-training model on dataset cricket_y
29/29 [==============================] - 0s 2ms/step
Pre-training model on dataset distal_phalanax_tw
29/29 [==============================] - 0s 3ms/step
Pre-training model on dataset distal_phalanx_outline
29/29 [==============================] - 0s 3ms/step
Pre-training model on dataset egg_five_days
29/29 [==============================] - 0s 3ms/step
Pre-training model on dataset electric_devices
29/29 [==============================] - 0s 3ms/step
Pre-training model on dataset face_ucr
29/29 [==============================] - 0s 3ms/step
Pre-training model on dataset fifty_words
29/29 [==============================] - 0s 3ms/step
Pre-training model on dataset gun_point_male_female
29/29 [==============================] - 0s 3ms/step
Pre-training model on dataset gun_point_old_young
29/29 [==============================] - 0s 3ms/step
Pre-training model on dataset large_kitchen_appliances
29/29 [========

In [9]:
results

[['cbf',
  'cricket_y',
  'Pre-Trained FCN (pre-trained on cricket_y; fine-tuned on cbf)',
  13.038226127624512,
  0.3355555534362793,
  '[[0, 0, 0, 1, 299], [0, 0, 0, 0, 298], [0, 0, 0, 0, 302], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]',
  '[0.24786324799060822, 0.3561253547668457, 0.4615384638309479, 0.470085471868515, 0.47863247990608215, 0.5441595315933228, 0.561253547668457, 0.5413105487823486, 0.5897436141967773, 0.6068376302719116, 0.621082603931427, 0.6125355958938599, 0.6381766200065613, 0.6609686613082886, 0.6723646521568298, 0.680911660194397, 0.7236467003822327, 0.692307710647583, 0.7094017267227173, 0.7094017267227173, 0.6980056762695312, 0.6980056762695312, 0.7293447256088257, 0.7150996923446655, 0.7321937084197998, 0.7293447256088257, 0.7435897588729858, 0.7293447256088257, 0.7407407164573669, 0.7236467003822327, 0.37037035822868347, 0.7037037014961243, 0.7037037014961243, 0.8518518805503845, 0.8518518805503845, 0.7777777910232544, 0.8888888955116272, 0.8518518805503845, 0.9629

In [10]:
df = pd.DataFrame(results, columns=['Target Dataset', 'Pre-Training Dataset', 'Model Name', 'Loss', 'Accuracy', 'Confusion Matrix', 'History'])
df

,Target Dataset,Pre-Training Dataset,Model Name,Loss,Accuracy,Confusion Matrix,History
0,cbf,cricket_y,Pre-Trained FCN (pre-trained on cricket_y; fin...,13.038226,0.335556,"[[0, 0, 0, 1, 299], [0, 0, 0, 0, 298], [0, 0, ...","[0.24786324799060822, 0.3561253547668457, 0.46..."
1,cbf,distal_phalanax_tw,Pre-Trained FCN (pre-trained on distal_phalana...,1.138480,0.636667,"[[0, 0, 300], [0, 0, 298], [0, 0, 302]]","[0.7111111283302307, 0.7388888597488403, 0.763..."
2,cbf,distal_phalanx_outline,Pre-Trained FCN (pre-trained on distal_phalanx...,17.364275,0.331111,"[[2, 298, 0], [10, 288, 0], [27, 275, 0]]","[0.6611111164093018, 0.7518518567085266, 0.775..."
3,cbf,egg_five_days,Pre-Trained FCN (pre-trained on egg_five_days;...,4.947879,0.360000,"[[0, 300, 0], [0, 298, 0], [0, 302, 0]]","[0.7945736646652222, 0.9354005455970764, 0.974..."
4,cbf,electric_devices,Pre-Trained FCN (pre-trained on electric_devic...,1.284791,0.695556,"[[0, 0, 0, 299, 1], [0, 0, 0, 271, 27], [0, 0,...","[0.7003610134124756, 0.7836424708366394, 0.808..."
...,...,...,...,...,...,...,...
267,synthetic_control,mote_strain,Pre-Trained FCN (pre-trained on mote_strain; f...,0.081079,0.980000,"[[50, 0, 0, 0, 0, 0], [50, 0, 0, 0, 0, 0], [50...","[0.915630578994751, 0.9431616067886353, 0.9404..."
268,synthetic_control,power_cons,Pre-Trained FCN (pre-trained on power_cons; fi...,0.039137,0.983333,"[[0, 50, 0, 0, 0, 0], [50, 0, 0, 0, 0, 0], [0,...","[0.8580247163772583, 0.8395061492919922, 0.858..."
269,synthetic_control,sony_robot,Pre-Trained FCN (pre-trained on sony_robot; fi...,0.266804,0.920000,"[[50, 0, 0, 0, 0, 0], [50, 0, 0, 0, 0, 0], [7,...","[0.9370370507240295, 0.9648148417472839, 0.988..."
270,synthetic_control,strawberry,Pre-Trained FCN (pre-trained on strawberry; fi...,0.244445,0.930000,"[[0, 50, 0, 0, 0, 0], [0, 50, 0, 0, 0, 0], [33...","[0.61524498462677, 0.6333938241004944, 0.67513..."


In [11]:
df.to_csv(CSV_PATH, index=False)